<b><img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250, style="padding: 10px"> 
<p><p><p><p><p><p>
<b>Playing with Images</b> <br>
Last verified to run on 2022-04-26 with LSST Science Pipelines release w_2022_12 <br>
Contact authors: Leanne Guy <br>
Target audience: All DP0 delegates. <br>
Minimum Container Size: medium <br>
Questions welcome at <a href="https://community.lsst.org/c/support/dp0">community.lsst.org/c/support/dp0</a> <br>
Find DP0 documentation and resources at <a href="https://dp0-1.lsst.io">dp0-1.lsst.io</a> <br>

### Learning Objective

A guided walk through Butler Gen3

References: https://github.com/lsst-dm/Nov19_bootcamp/blob/main/notebooks/Gen3Butler.ipynb

### Setup 

In [ ]:
# General python imports
import numpy as np
import warnings

# Astropy
from astropy.visualization import  ZScaleInterval, AsinhStretch

# LSST imports
from lsst.daf.butler import CollectionType, DatasetType, Butler

# Bokeh and Holoviews for visualization
import bokeh
from bokeh.io import output_notebook
from bokeh.models import HoverTool

import holoviews as hv
from holoviews import streams, opts
from holoviews.operation.datashader import rasterize

# Set the holoviews plotting library to be bokeh
# You will see the holoviews + bokeh icons displayed when the library is loaded successfully
hv.extension('bokeh')

# Display bokeh plots inline in the notebook
output_notebook()

In [ ]:
### Instantiate a Butler to work with DP0.1

In [ ]:
# Instantiate the Butler initializing it with the repository name and the DP0.1 collection identifier
from lsst.daf.butler import Butler
repo = 's3://butler-us-central1-dp01'
collection = '2.2i/runs/DP0.1'
butler = Butler(repo, collections=collection)

#### Start with a coadded image 


In [ ]:
coaddId = {'tract': 4226, 'patch': 17, 'band': 'r'}
coadd = butler.get('deepCoadd', coaddId)
assert coadd is not None

In [98]:
import shutil, os, yaml, re
from lsst.daf.butler import CollectionType, DatasetType, Butler, ButlerConfig

In [79]:
testCaseName = "LVV-T2476"
baseDir = "/home/leannep/WORK"
testDir = os.path.join(baseDir, testCaseName)
if os.path.exists(testDir):
    shutil.rmtree(testDir)
os.makedirs(testDir)

In [80]:
repo = os.path.join(testDir, "repo/default")
config = Butler.makeRepo(repo)

In [81]:
butler = Butler(repo, writeable=True)
butler.registry.registerCollection("c", CollectionType.RUN) is True
dataset_type = DatasetType("dt", (), "StructuredDataDict", universe=butler.registry.dimensions)
butler.registry.registerDatasetType(dataset_type)
butler.put({"key": "value"}, dataset_type, {}, run="c")
print(butler.getURI(dataset_type, {}, collections="c"))

file:///home/leannep/WORK/LVV-T2476/repo/default/c/dt/dt_c.yaml


In [89]:
uri=butler.getURI(dataset_type, {}, collections="c")

In [99]:
print(uri.geturl())

file:///home/leannep/repos/lsst-notebooks/repo/json/c/dt/dt_c.json


In [102]:
x = re.search('json$', uri.geturl())
print(x)

<re.Match object; span=(62, 66), match='json'>


In [82]:
config = ButlerConfig(repo)

In [83]:
d = config.toDict()
d['datastore']['formatters']['StructuredDataDict'] = 'lsst.daf.butler.formatters.json.JsonFormatter'

In [84]:
print(d['datastore']['formatters']['StructuredDataDict'])

lsst.daf.butler.formatters.json.JsonFormatter


In [87]:
with open('test.yaml', 'w') as outStream:
    yaml.safe_dump(d, outStream, default_flow_style=False, explicit_start=True)

In [86]:
repo_json = "repo/json"
assert Butler.makeRepo(repo_json, config="test.yaml") is not None

FileExistsError: [Errno 17] File exists: '/home/leannep/repos/lsst-notebooks/repo/json/butler.yaml'

In [88]:
butler = Butler(repo_json, writeable=True)
butler.registry.registerCollection("c", CollectionType.RUN)

False

In [62]:
dataset_type = DatasetType("dt", (), "StructuredDataDict", universe=butler.registry.dimensions)
butler.registry.registerDatasetType(dataset_type)

False

In [103]:
butler.put({"key": "value"}, dataset_type, {}, run="c")
uri = butler.getURI(dataset_type, {})

ConflictingDefinitionError: A database constraint failure was triggered by inserting one or more datasets of type DatasetType('dt', {}, StructuredDataDict) into collection 'c'. This probably means a dataset with the same data ID and dataset type already exists, but it may also mean a dimension row is missing.

In [104]:
type(uri)

lsst.resources.file.FileResourcePath